In [1]:
import requests
import mysql.connector
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Connect to MySQL
cnx = mysql.connector.connect(user='your_username', password='your_password', host='localhost', database='firearms_data')
cursor = cnx.cursor()

ProgrammingError: 1045 (28000): Access denied for user 'your_username'@'localhost' (using password: YES)

In [ ]:
# Retrieve data from link
url = 'https://erich-springer.livejournal.com/22885.html'
response = requests.get(url)
content = response.content.decode('utf-8')


In [ ]:
# Parse data using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
# Extract firearms data
firearms_data = []
for tr in soup.find_all('tr')[1:]:
  tds = tr.find_all('td')
  anime_character = tds[0].text.strip()
  firearm_name = tds[1].text.strip()
  firearm_type = tds[2].text.strip()
  firearm_manufacturer = tds[3].text.strip()
  firearm_caliber = tds[4].text.strip()
  firearm_capacity = int(tds[5].text.strip())
  firearm_length = float(tds[6].text.strip().replace('in', '').replace('"', ''))
  firearm_weight = float(tds[7].text.strip().replace('oz', '').replace('lb', ''))
  firearm_features = tds[8].text.strip()
  firearms_data.append((anime_character, firearm_name, firearm_type, firearm_manufacturer, firearm_caliber, firearm_capacity, firearm_length, firearm_weight, firearm_features))


In [ ]:
# Insert data into MySQL
insert_query = '''
INSERT INTO firearms (anime_character, firearm_name, firearm_type, firearm_manufacturer, firearm_caliber, firearm_capacity, firearm_length, firearm_weight, firearm_features)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
'''
cursor.executemany(insert_query, firearms_data)
cnx.commit()

In [ ]:
# Query firearms data from MySQL
query = '''
SELECT anime_character, firearm_name, firearm_type, firearm_manufacturer, firearm_caliber, firearm_capacity, firearm_length, firearm_weight, firearm_features FROM firearms
'''
cursor.execute(query)

# Fetch data as a list of tuples
data = cursor.fetchall()



In [ ]:
# Convert data to Pandas DataFrame
df = pd.DataFrame(firearms_data, columns=[
    'anime_character',
    'firearm_name',
    'firearm_type',
    'firearm_manufacturer',
    'firearm_caliber',
    'firearm_capacity',
    'firearm_length',
    'firearm_weight',
    'firearm_features'
])


In [ ]:
# Close MySQL connection
cursor.close()
cnx.close()